In [1]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error as mae

from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score

In [3]:
train_one = pd.read_csv("data/train4.csv")
test_one = pd.read_csv("data/test4.csv")

In [4]:
X = train_one.drop(['등록차량수','단지코드'],axis=1)
y = train_one['등록차량수']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=2021,shuffle=True)

cb = CatBoostRegressor(random_state=2021,verbose=False,loss_function='MAE').fit(X_train, y_train)
pred = cb.predict(X_test)

mae(pred,y_test) 

119.87924518994078

In [6]:
real_x = test_one.drop('단지코드',axis=1)
real_y = cb.predict(real_x)

In [7]:
def myfunc():
    cb = pd.read_csv("0713_cb.csv")
    cb = cb.loc[-cb['code'].isin(['C2675', 'C2335', 'C1327']),:].reset_index(drop=True)
    return cb

res = myfunc()
mae(real_y,res['num'])

55.62185196973271

In [8]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")

test_one['num'] = real_y
res = test_one[['단지코드','num']]

sub = pd.read_csv(os.path.join(path,"sample_submission.csv"))
sub['num'] = 0
sub = pd.merge(sub,res,left_on='code',right_on='단지코드',how='left').drop(['num_x','단지코드'],axis=1)

sub = sub.rename(columns={'num_y':'num'})

sub.to_csv("sub3.csv",index=False)